# Тестовое задание Спектрум

## Сохранение исходных данных в БД

In [4]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from sqlalchemy import create_engine
import os

In [5]:
def download_from_google_sheets(credentials_file, spreadsheet_id, sheet_name=None):
    """
    Загружает данные из Google Sheets и возвращает DataFrame
    
    Args:
        credentials_file (str): Путь к JSON файлу с ключами сервисного аккаунта
        spreadsheet_id (str): ID Google таблицы
        sheet_name (str): Название листа (если None, берется первый лист)
    
    Returns:
        pd.DataFrame: DataFrame с данными из таблицы
    """
    # Настройки Google Sheets API
    SCOPE = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    
    try:
        # Аутентификация в Google Sheets
        credentials = Credentials.from_service_account_file(credentials_file, scopes=SCOPE)
        gc = gspread.authorize(credentials)
        
        # Открытие таблицы
        spreadsheet = gc.open_by_key(spreadsheet_id)
        
        # Выбор листа
        if sheet_name:
            worksheet = spreadsheet.worksheet(sheet_name)
        else:
            worksheet = spreadsheet.sheet1
        
        # Чтение данных
        data = worksheet.get_all_values()
        
        if not data:
            print("Таблица пустая")
            return pd.DataFrame()
        
        headers = data[0]  # первая строка - заголовки
        rows = data[1:]    # остальные строки - данные
        
        df = pd.DataFrame(rows, columns=headers)
        
        # Очистка данных
        df = df.replace('', pd.NA)
        df = df.dropna(how='all')
        
        print(f"Успешно загружено {len(df)} строк из Google Sheets")
        return df
        
    except Exception as e:
        print(f"Ошибка при загрузке из Google Sheets: {e}")
        return pd.DataFrame()


def save_to_excel(df, file_path, sheet_name='Data'):
    """
    Сохраняет DataFrame в Excel файл
    
    Args:
        df (pd.DataFrame): DataFrame для сохранения
        file_path (str): Путь для сохранения Excel файла
        sheet_name (str): Название листа в Excel
    
    Returns:
        bool: True если успешно, False если ошибка
    """
    try:
        if df.empty:
            print("DataFrame пустой, нечего сохранять")
            return False
        
        # Сохранение в Excel
        df.to_excel(file_path, sheet_name=sheet_name, index=False)
        print(f"Данные успешно сохранены в {file_path}")
        return True
        
    except Exception as e:
        print(f"Ошибка при сохранении в Excel: {e}")
        return False


def save_to_postgres(df, user, password, host, port, db_name, table_name):
    """
    Сохраняет DataFrame в PostgreSQL
    
    Args:
        df (pd.DataFrame): DataFrame для сохранения
        user (str): Имя пользователя PostgreSQL
        password (str): Пароль PostgreSQL
        host (str): Хост PostgreSQL
        port (str): Порт PostgreSQL
        db_name (str): Имя базы данных
        table_name (str): Имя таблицы
    
    Returns:
        bool: True если успешно, False если ошибка
    """
    try:
        if df.empty:
            print("DataFrame пустой, нечего сохранять")
            return False
        
        # Подключение к PostgreSQL
        engine = create_engine(
            f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}'
        )
        
        # Сохранение в PostgreSQL
        df.to_sql(
            table_name,
            engine,
            if_exists='replace',
            index=False,
            method='multi'
        )
        
        print(f"Успешно сохранено {len(df)} строк в таблицу {table_name}")
        return True
        
    except Exception as e:
        print(f"Ошибка при сохранении в PostgreSQL: {e}")
        return False

In [7]:
CREDENTIALS_FILE = 'spectrum-test-472019-3dcf8d526f89.json'
SPREADSHEET_ID = '1qkjfZ-3IgNLxmIXmYfWdJ0NK3eW_eQLqvabzimeHCNo'
EXCEL_FILE_PATH = 'data_from_google.xlsx'
    
# Загрузка данных из Google Sheets
df = download_from_google_sheets(CREDENTIALS_FILE, SPREADSHEET_ID)
    
if not df.empty:
    # Показать информацию о данных
    print("Информация о данных:")
    print(f"Размер: {df.shape}")
    print(f"Столбцы: {list(df.columns)}")
    print("\nПервые 5 строк:")
    print(df.head())
        
    # Сохранение в Excel
    save_to_excel(df, EXCEL_FILE_PATH)
        
    # Опционально: сохранение в PostgreSQL
    # save_to_postgres(
    #     df=df,
    #     user='postgres',
    #     password='s3cret',
    #     host='localhost',
    #     port='5432',
    #     db_name='spectrum',
    #     table_name='raw_table'
    # )

Успешно загружено 131 строк из Google Sheets
Информация о данных:
Размер: (131, 10)
Столбцы: ['№ площадки', '№ объекта', 'Марка', 'Шифр комплекта', 'Стоимость', 'Начало выполнения', 'Завершение выполнения', 'Подрядчик', 'Стоимость подрядчик 1', 'Стоимость подрядчик 2']

Первые 5 строк:
  № площадки № объекта Марка       Шифр комплекта      Стоимость  \
0          1        К4    КЖ   2025-07/01-К4-КЖ11   6 928 512,00   
1          1        К4    КМ  2025-07/01-К4-КМ8.2  27 941 657,10   
2          1        К4    СС    2025-07/01-К4-СС1   4 112 072,52   
3          1        К4    СС    2025-07/01-К4-СС2   4 112 072,52   
4          1        К4    СС    2025-07/01-К4-СС3   4 112 072,52   

  Начало выполнения Завершение выполнения    Подрядчик Стоимость подрядчик 1  \
0        07.04.2025            05.10.2025  подрядчик 1          6 928 512,00   
1        28.07.2025            16.11.2025  подрядчик 1         27 941 657,10   
2        01.12.2025            15.03.2026  подрядчик 1          